# Object Detection with RetinaNet

**Author:** [Srihari Humbarwadi](https://twitter.com/srihari_rh)<br>
**Date created:** 2020/05/17<br>
**Last modified:** 2023/07/10<br>
**Description:** Implementing RetinaNet: Focal Loss for Dense Object Detection.


## Introduction

Object detection a very important problem in computer
vision. Here the model is tasked with localizing the objects present in an
image, and at the same time, classifying them into different categories.
Object detection models can be broadly classified into "single-stage" and
"two-stage" detectors. Two-stage detectors are often more accurate but at the
cost of being slower. Here in this example, we will implement RetinaNet,
a popular single-stage detector, which is accurate and runs fast.
RetinaNet uses a feature pyramid network to efficiently detect objects at
multiple scales and introduces a new loss, the Focal loss function, to alleviate
the problem of the extreme foreground-background class imbalance.

**References:**

- [RetinaNet Paper](https://arxiv.org/abs/1708.02002)
- [Feature Pyramid Network Paper](https://arxiv.org/abs/1612.03144)


In [1]:
import os
import re
import zipfile

os.environ["CUDA_VISIBLE_DEVICES"] = "0"

import numpy as np
import tensorflow as tf
from tensorflow import keras

import matplotlib.pyplot as plt
import tensorflow_datasets as tfds

2024-12-07 10:33:03.321307: I tensorflow/core/util/port.cc:111] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-12-07 10:33:03.362712: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-12-07 10:33:03.362738: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-12-07 10:33:03.362765: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-12-07 10:33:03.370538: I tensorflow/core/platform/cpu_feature_g

In [2]:
physical_gpus = tf.config.list_physical_devices("GPU")
tf.config.set_logical_device_configuration(
    physical_gpus[0], [tf.config.LogicalDeviceConfiguration(memory_limit=30000)]
)

## Downloading the COCO2017 dataset


Training on the entire COCO2017 dataset which has around 118k images takes a lot of time, hence we will be using a smaller subset of ~500 images for training in this example.


In [3]:
url = "https://github.com/srihari-humbarwadi/datasets/releases/download/v0.1.0/data.zip"
filename = os.path.join(os.getcwd(), "data.zip")
keras.utils.get_file(filename, url)


with zipfile.ZipFile("data.zip", "r") as z_fp:
    z_fp.extractall("./")

560525318/560525318 [==============================] - 42s 0us/step


## Implementing utility functions

Bounding boxes can be represented in multiple ways, the most common formats are:

- Storing the coordinates of the corners `[xmin, ymin, xmax, ymax]`
- Storing the coordinates of the center and the box dimensions
  `[x, y, width, height]`

Since we require both formats, we will be implementing functions for converting
between the formats.


In [ ]:
def swap_xy(boxes):
    """Swaps the order of x and y coordinates of the boxes. While a tensor shape
    is represented as (height, width), that of an image is represented as
    (width, height). So this function will help.

    Arguments:
        boxes: A tensor with shape `(num_boxes, 4)` representing bounding boxes.

    Returns:
        swapped boxes with shape same as that of boxes
    """
    return tf.stack([boxes[:, 1], boxes[:, 0], boxes[:, 3], boxes[:, 2]], axis=-1)


def convert_to_xywh(boxes):
    """Changes the box format to center, width and height

    Arguments:
        boxes: A tensor of rank 2 or higher with a shape of `(..., num_boxes, 4)`
        representing bounding boxes where each box is of the format
        `[xmin, ymin, xmax, ymax]`

    Returns
        converted boxes with shape same as that of boxes
    """
    return tf.concat(
        [(boxes[..., :2] + boxes[..., 2:]) / 2, boxes[..., 2:] - boxes[..., :2]],
        axis=-1,
    )


def convert_to_corners(boxes):
    """Changes the box format to corner coordinates

    Arguments:
        boxes: A tensor of rank 2 or higher with a shape of `(..., num_boxes, 4)`
        representing bounding boxes where each box is of the format
        `[x, y, width, height]`

    Returns
        converted boxes with shape same as that of boxes
    """
    return tf.concat(
        [boxes[..., :2] - boxes[..., 2:] / 2, boxes[..., :2] + boxes[..., 2:] / 2],
        axis=-1,
    )

## Computing pairwise Intersection Over Union (IOU)

As we will see later in the example, we would be assigning ground truth boxes
to anchor boxes based on the extent of overlapping. This will require us to
calculate the Intersection Over Union (IOU) between all the anchor
boxes and ground truth boxes pairs.


In [4]:
def compute_iou(boxes1, boxes2):
    """Computes pairwise IOU matrix for given two sets of boxes

    Arguments:
        boxes1: A tensor with shape `(N, 4)` representing bounding boxes where
        each box is of the format `[x, y, width, height]`.
        boxes2: A tensor with shape `(M, 4)` representing bounding boxes where
        each box is of the format `[x, y, width, height]`.

    Returns:
        pairwise IOU matrix with shape `(N, M)`, where the value at i-th row,
        j-th column holds the IOU between i-th box and j-th box from boxes1 and
        boxes2 respectively.
    """
    boxes1_corners = convert_to_corners(boxes1)
    boxes2_corners = convert_to_corners(boxes2)

    max_upper_left = tf.maximum(  # shape (N, M, 2)
        boxes1_corners[:, None, :2],
        boxes2_corners[:, :2],
    )
    min_lower_right = tf.minimum(  # shape (N, M, 2)
        boxes1_corners[:, None, 2:],
        boxes2_corners[:, 2:],
    )

    intersection = tf.maximum(0.0, min_lower_right - max_upper_left)  # shape (N, M, 2)
    intersection_area = tf.reduce_prod(intersection, axis=-1)  # shape (N, M)

    boxes1_area = tf.reduce_prod(boxes1[:, 2:], axis=-1)  # shape (N, )
    boxes2_area = tf.reduce_prod(boxes2[:, 2:], axis=-1)  # shape (M, )

    union_area = boxes1_area[:, None] + boxes2_area - intersection_area
    return intersection_area / union_area


def visualize_detections(
    image, boxes, classes, scores, figsize=(7, 7), linewidth=7, color=[0, 0, 1]
):
    """Visualize Detections"""
    image = np.array(image, dtype=np.uint8)
    plt.figure(figsize=figsize)
    plt.axis("off")
    plt.imshow(image)
    ax = plt.gca()

    for box, _cls, score in zip(boxes, classes, scores):
        text = f"{_cls}: {score:.2f}"
        x1, y1, x2, y2 = box
        w, h = x2 - x1, y2 - y1
        patch = plt.Rectangle(
            [x1, y1], w, h, fill=False, edgecolor=color, linewidth=linewidth
        )
        ax.add_patch(patch)
        ax.text(
            x1,
            y1,
            text,
            bbox={"facecolor": color, "alpha": 0.4},
            clip_box=ax.clipbox,
            clip_on=True,
        )
    plt.show()
    return ax

## Implementing Anchor generator

Anchor boxes are fixed sized boxes that the model uses to predict the bounding box for an object. It does this by regressing the offset between the location of the object's center and the center of an anchor box, and then uses the width and height of the anchor box to predict a relative scale of the object. In the case of RetinaNet, each location on a given feature map has nine anchor boxes (at three scales and three ratios).


In [11]:
class AnchorBox:
    """Generates anchor boxes

    This class has operations to generate anchor boxes for feature maps at
    strides `[8, 16, 32, 64, 128]`. Each anchor box is of the format
    `[x, y, width, height]`

    Attributes:
        `aspect_ratios`: A list of float values representing the aspect ratios of
        the anchor boxes at each location on the feature map.
        `scales`: A list of float values representing the scale of the anchor boxes
        at each location on the feature map.
        `num_anchors`: The number of anchor boxes at each location on the feature
        map.
        `areas`: A list of float vales representing the areas of the anchor boxes
        for each feature map in the feature pyramid.
        `strides`: A list of float values representing the strides for each
        feature map in the feature pyramid.
    """

    def __init__(self):
        self.aspect_ratios = [0.5, 1.0, 2.0]  # aspect ratio = width / height
        self.scales = [2**x for x in [0, 1 / 3, 2 / 3]]
        self.num_anchors = len(self.aspect_ratios) * len(self.scales)

        self.strides = [2**i for i in range(3, 8)]
        self.areas = [x**2 for x in [32, 64, 128, 256, 512]]
        self.anchor_dims = self.compute_dims()

    def compute_dims(self):
        """Computes anchor box dimensions for all ratios and scales at all levels
        of the feature pyramid."""
        # This list will have 5 tensor of shape (1, 1, 9, 2)
        anchor_dims_all = []
        for area in self.areas:
            anchor_dims = []

            for ratio in self.aspect_ratios:
                anchor_height = tf.math.sqrt(area / ratio)  # h.w = S <=> h.h.r = S
                anchor_width = area / anchor_height

                dims = tf.reshape(
                    tf.stack([anchor_width, anchor_height], 0),
                    shape=[1, 1, 2],
                )

                # After the loop, anchor_dims will receive 3 more tensor of shape (1, 1, 2)
                for scale in self.scales:
                    anchor_dims.append(scale * dims)
            # At this point, anchor_dims will have 9 tensor of shape (1, 1, 2)

            # anchor_dims will be converted to tensor of shape (1, 1, 9, 2)
            anchor_dims_all.append(tf.stack(anchor_dims, axis=-2))

        return anchor_dims_all  # array len 5

    def _get_anchors(self, feature_height, feature_width, level):
        """Generates anchor boxes for a given feature map size and level

        Arguments:
            feature_height: An integer representing the height of the feature map
            feature_width: An integer representing the width of the feature map
            level: An integer representing the level of the feature map in the
            feature pyramid

        Returns
            anchor boxes with the shape of
            `(feature_height * feature_width * num_anchors, 4)`
        """
        rx = tf.range(feature_width, dtype=tf.float32) + 0.5
        ry = tf.range(feature_height, dtype=tf.float32) + 0.5

        centers = tf.stack(tf.meshgrid(rx, ry), axis=-1)  # shape (height, width, 2)
        centers *= self.strides[level]
        centers = tf.expand_dims(centers, axis=-2)  # shape (height, width, 1, 2)
        centers = tf.tile(
            centers,
            [1, 1, self.num_anchors, 1],
        )  # shape (height, width, 9, 2)

        dims = tf.tile(
            self.anchor_dims[level],
            [feature_height, feature_width, 1, 1],
        )  # shape (height, width, 9, 2)

        # Each pixel (element) in the feature map will have 9 anchors and each
        # anchor is represented by 4 values corresponding to its yc, xc, w, h
        anchors = tf.concat([centers, dims], axis=-1)  # shape (height, width, 9, 4)
        anchors = tf.reshape(
            anchors,
            [feature_height * feature_width * self.num_anchors, 4],
        )
        return anchors

    def get_anchors(self, image_height, image_width):
        """Generates anchor boxes for all the feature maps of the feature pyramid.

        Arguments:
            image_height: Height of the input image
            image_width: Width of the input image

        Returns
            anchor boxes for all the feature maps, stacked as a single tensor
            with shape of `(total_anchors, 4)`
        """
        anchors = [
            self._get_anchors(
                tf.math.ceil(image_height / stride),
                tf.math.ceil(image_width / stride),
                i,
            )
            for i, stride in enumerate(self.strides)
        ]
        anchors = tf.concat(anchors, axis=0)
        return anchors

## Preprocessing data

Preprocessing the images involves two steps:

- Resizing the image: Images are resized such that the shortest size is equal
to 800 px, after resizing if the longest side of the image exceeds 1333 px,
the image is resized such that the longest size is now capped at 1333 px.
- Applying augmentation: Random scale jittering  and random horizontal flipping
are the only augmentations applied to the images.

Along with the images, bounding boxes are rescaled and flipped if required.

In [18]:
def random_flip_horizontal(image, boxes):
    """Flips image and boxes horizontally with 50% chance.

    Arguments:
        image: A 3-D tensor of shape `(height, width, channels)` representing
        an image.
        boxes: A tensor of shape `(num_boxes, 4)` representing bounding boxes
        with normalized `(xmin, ymin, xmax, ymax)` format.

    Returns:
        Randomly flipped image and boxes.
    """
    if tf.random.uniform([]) > 0.5:
        image = tf.image.flip_left_right(image)
        boxes = tf.stack(
            [1 - boxes[:, 2], boxes[:, 1], 1 - boxes[:, 0], boxes[:, 3]],
            axis=-1,
        )
    return image, boxes


def resize_and_pad_image(
    image, min_side=800, max_side=1333, jitter=[640, 1024], stride=128
):
    """Resizes and pads an image while preserving aspect ratio.

    1. Resizes an image so that the shorter side is equal to `min_side`.
    2. If the longer side is greater than `max_side`, then resize the image with
    longer side equal to `max_side`.
    3. Pad with zeros on right and bottom to make the image shape divisible by
    `stride`.

    Arguments:
        image: A 3-D tensor of shape `(height, width, channels)` representing an
        image.
        min_side: The shorter side of the image is resized to this value, if
        `jitter` is set to None.
        max_side: If the longer side of the image exceeds this value after
        resizing, the image is resized such that the longer side now equals to
        this value.
        jitter: A list of floats containing minimum and maximum size for scale
        jittering. If available, the shorter side of the image will be resized to
        a random value in this range.
        stride: The stride of the smallest feature map in the feature pyramid.
        Can be calculated using `image_size / feature_map_size`.

    Returns:
        image: Resized and padded image.
        image_shape: Shape of image before padding.
        scaling_factor: Value used to resize image.
    """
    image_shape = tf.cast(tf.shape(image)[:2], dtype=tf.float32)

    if jitter:
        min_side = tf.random.uniform(
            [], minval=jitter[0], maxval=jitter[1], dtype=tf.float32
        )

    # Compute scaling factor
    scaling_factor = min_side / tf.reduce_min(image_shape)
    if scaling_factor * tf.reduce_max(image_shape) > max_side:
        scaling_factor = max_side / tf.reduce_max(image_shape)

    # Compute new image shape for resizing
    image_shape *= scaling_factor
    image = tf.image.resize(image, tf.cast(image_shape, dtype=tf.int32))

    padded_image_shape = tf.cast(
        tf.math.ceil(image_shape / stride) * stride,
        dtype=tf.int32,
    )
    image = tf.image.pad_to_bounding_box(
        image, 0, 0, padded_image_shape[0], padded_image_shape[1]
    )
    return image, image_shape, scaling_factor


def preprocess_data(sample):
    """Applies preprocessing step to a single sample.

    Arguments:
        sample: A dict representing a single training sample.

    Returns:
        image: Resized and padded image with random horizontal flipping
        applied.
        bbox: bounding boxes with the shape `(num_objects, 4)` where each box
        is of the format `[x, y, width, height]`.
        class_id: An tensor representing the class ids of the objects, having
        shape `(num_objects,)`.
    """
    image = sample["image"]
    bbox = swap_xy(sample["objects"]["bbox"])
    class_id = tf.cast(sample["objects"]["label"], dtype=tf.int32)

    image, bbox = random_flip_horizontal(image, bbox)
    image, image_shape, _ = resize_and_pad_image(image)

    bbox = tf.stack(
        [
            bbox[:, 0] * image_shape[1],
            bbox[:, 1] * image_shape[0],
            bbox[:, 2] * image_shape[1],
            bbox[:, 3] * image_shape[0],
        ],
        axis=-1,
    )
    bbox = convert_to_xywh(bbox)
    return image, bbox, class_id

## Encoding labels

The raw labels, consisting of bounding boxes and class ids, need to be
transformed into targets for training. This transformation consists of
the following steps:

- Generating anchor boxes for the given image dimensions
- Assigning ground truth boxes to the anchor boxes
- The anchor boxes that are not assigned any objects, are either assigned the
  background class or ignored depending on the IOU
- Generating the classification and regression targets using anchor boxes
